In [13]:
import torch
import timm

from src.util.sam import SAM
from pytorch_lightning import LightningModule

In [14]:
data_dir = "./data"

In [29]:
class GravModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('tf_efficientnet_l2_ns_475', pretrained=True)

    def configure_optimizers(self):
        base_optimizer = torch.optim.Adam
        optimizer = SAM(self.parameters(), base_optimizer, lr=0.1, momentum=0.9)
        return optimizer
    
    def forward(self, x):
        x = 0
        return self.model(x)